In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#load train file list, valid file list and test file list
path = {}
path['input_data'] = ['/kaggle/input/prepare-ecg-reg-data']

def get_train_val_test_list(paths):
    files = []
    for dir_add in paths:
        f_adds = os.listdir(dir_add)
        f_adds = sorted(f_adds)
        for i,f_add in enumerate(f_adds.copy()):
            if '.part' not in f_add:
                f_adds.remove(f_add)
            else:
                f_add = dir_add + '/' + f_add
                files.append(f_add)
    num_samples = len(files)
    num_valids = 0
    num_tests = 543
    num_trains = num_samples - num_valids - num_tests
    #return train_list, valid_list, test_list
    return files
    
file_adds = get_train_val_test_list(path['input_data'])

In [ ]:
import base64
def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    sn = np.where(sd == 0, 0, m/sd)
    return 10*(np.log(sn)/np.log(10))

def computeSTN(data):
    stn = [signaltonoise(item) for item in data]
    return sum(stn)/len(stn)

def parse_line(f_add):
    
    with open(f_add, 'r') as f:
        line = f.read()
        id, name, segment, data, label, features = line.strip().split('\t')
        
        index_data = base64.b64decode(data)
        index_data = np.fromstring(index_data).reshape(12,-1)
        
        index_label = base64.b64decode(label)
        index_label = np.fromstring(index_label)
        
        sample = {}
        sample['id'] = id
        sample['file_name'] = f_add
        sample['index_name'] = name
        sample['segment'] = segment
        sample['index_data'] = index_data
        sample['index_label'] = index_label
        sample['features'] = features
        
        return sample

In [ ]:
raw_datas = []
for f_add in tqdm(file_adds):
    sample = parse_line(f_add)
    raw_datas.append(sample['index_data'])

In [ ]:
import pywt
basewavelet = 'coif5'
w = pywt.Wavelet(basewavelet)
maxlev = pywt.dwt_max_level(len(data), w.dec_len)
# maxlev = 2 # Override if desired
print("maximum level is " + str(maxlev))
threshold = 0.04 # Threshold for filtering

# Decompose into wavelet components, to the level selected:
coeffs = pywt.wavedec(data, basewavelet, level=maxlev)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
for i in range(1, len(coeffs)):
    plt.subplot(maxlev, 1, i)
    plt.plot(coeffs[i])
    coeffs[i] = pywt.threshold(coeffs[i], threshold*max(coeffs[i]))
    plt.plot(coeffs[i])
datarec = pywt.waverec(coeffs, basewavelet)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(raw_datas[0][0])

In [ ]:
plt.plot(datarec)

In [ ]:
import pywt
import pylab
[phi, psi, x] = pywt.Wavelet('coif5').wavefun(level=5)
pylab.plot(x, psi)
pylab.title('Coiflet-5')
pylab.xlabel('Time')
#pylab.show()
pylab.savefig('temp.png', transparent=True)

In [ ]:
from IPython.display import FileLink
FileLink('temp.png')